In [ ]:
%load_ext autoreload
%autoreload 2
#Global Configurations
import sys
sys.path.append('/opt/ros/noetic/lib/python3/dist-packages/')
global_rosbag_path = '/home/xuhao/data/camera190_closer/camera190_closer.bag'
global_output_path = '/home/xuhao/data/camera190_closer/'

global_photometric_calibration_path = None

global_image_topic = '/oak_ffc_4p/assemble_image/compressed'
global_imu_topic = '/uav3/mavros/imu/data_raw'

global_virtual_fov = 190
global_virtual_width = 320
global_virtual_height = 240

In [ ]:
# Split bag into 9 parts for calibration maybe slow if bag is large
import utils.Utils as Utils
import utils.BagExtractor as BagExtractor

ret = Utils.check_rosbag(global_imu_topic,global_image_topic,global_rosbag_path)
if ret == False:
  exit(1)
bags = BagExtractor.extract_calibration_bag(global_rosbag_path,global_output_path, global_image_topic)
print("[Bag extractor] The following bags are created:\n")
for bag_file in bags:
  print(bag_file)

In [ ]:
# Calibrate each fisheye camera 
import utils.Calibration as Calibration

Calibration.calibrate_monos(global_output_path)
Calibration.check_calibration(global_output_path,"mono")

In [ ]:
# # Result of this cell is temporary unused in the following cells
# import utils.Calibration as Calibration

# Calibration.calibrate_stereos(global_output_path)
# Calibration.check_calibration(global_output_path,"stereo")

In [ ]:
import utils.Calibration as Calibration
Calibration.calibrate_cam_imu(global_output_path)
Calibration.check_imu_calibration(global_output_path)

In [ ]:
import utils.Utils as Utils
Utils.generate_d2vins_cinfiguration(global_output_path)

In [ ]:
import utils.VirtualStereoCalibration as VirtualStereoCalibration
input_bag = global_output_path + "/stereo_depth_calibration.bag"
output_path = global_output_path + f"/virtual_stereo_calibration_{global_virtual_fov}"
fisheye_config = global_output_path+ "fisheye_cams.yaml"
step = 1 
VirtualStereoCalibration.calibrate_virtual_stereo(input_bag,  global_virtual_fov, 
                                                  global_virtual_width, global_virtual_height,
                                                  fisheye_config, output_path, step, global_photometric_calibration_path,verbose=False)